In [1]:
from mewpy.simulation import solvers
from mewpy.simulation import set_default_solver
set_default_solver('glpk')

In [2]:
from cobra.io.sbml import read_sbml_model
model = read_sbml_model('Human-GEM-annotated.xml')
model

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


Name,HumanGEM
Memory address,0x0119a59ee0
Number of metabolites,8400
Number of reactions,13096
Number of groups,143
Objective expression,1.0*biomass_human - 1.0*biomass_human_reverse_fb2f2
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


In [6]:
def upstreamReactions(reaction_id, printResult = True):
    if printResult == True:
        print('target reaction',':',model.reactions.get_by_id(reaction_id).id,':',model.reactions.get_by_id(reaction_id).name)
        print('\n')
        metabolites_list = []
        immediately_connected = {}
        connected_by_one_reaction = {}
        do_not_scan = ['H2O','Pi','H+','PPi','NADPH','NADP+'] #append this with whatever you aren't interested in
        for m in (model.reactions.get_by_id(reaction_id).metabolites):
            print('metabolite:',m.name)
            rxns = []
            for rxn in m.reactions:
                print(m.name,'->',rxn.id,':',rxn.name)
                rxns.append(rxn.id)
                metabolites_list.append(m.name)
            immediately_connected[m.name] = rxns
            print('\n')
        print('reactions immediately connected to the target reaction:')
        print(rxns)
        print('\n')
        print('reactions connected to immediately connected reactions:')
        for r in rxns:
            for mb in (model.reactions.get_by_id(r).metabolites):
                rxns_2 = []
                if mb.name not in metabolites_list:
                    if mb.name not in do_not_scan:
                        print('metabolite:',mb.name)
                        metabolites_list.append(mb.name)
                        for reaction in mb.reactions:
                            print(mb.name,'->',reaction.id,':',reaction.name)
                            rxns_2.append(reaction.id)
                        connected_by_one_reaction[mb.name] = rxns_2
                        print('\n')
        return immediately_connected, connected_by_one_reaction

In [11]:
#example of saving function results as dictionary variables
immediately_connected, connected_by_one_reaction = upstreamReactions('HMR_5401',printResult = True)

target reaction : HMR_5401 : -1.0 glycogenin G7 + -7.0 Pi --> 7.0 glucose-1-phosphate + 1.0 glycogenin


metabolite: glycogenin G7
glycogenin G7 -> HMR_5400 : -1.0 glycogenin G7G1 + -1.0 H2O --> 1.0 glucose + 1.0 glycogenin G7
glycogenin G7 -> HMR_5401 : -1.0 glycogenin G7 + -7.0 Pi --> 7.0 glucose-1-phosphate + 1.0 glycogenin
glycogenin G7 -> GLDBRAN : -1.0 glycogenin G4G4 + -1.0 H2O --> 1.0 glucose + 1.0 glycogenin G7


metabolite: Pi
Pi -> HMR_4064 : -1.0 H2O + -1.0 pyridoxal-phosphate --> 1.0 Pi + 1.0 pyridoxal
Pi -> HEXDIACATP : -1.0 ATP + -1.0 H2O --> 1.0 Hexadecanediocacid + 1.0 ADP + 1.0 H+ + 1.0 Pi
Pi -> HMR_5166 : -801.0 ATP + -11.0 glycyl-tRNA(gly) + -802.0 H2O + -23.0 L-alanyl-tRNA(ala) + -17.0 L-arginyl-tRNA(arg) + -5.0 L-asparaginyl-tRNA(asn) + -16.0 L-aspartyl-tRNA(asp) + -19.0 L-glutaminyl-tRNA(gln) + -30.0 L-glutamyl-tRNA(glu) + -6.0 L-histidyl-tRNA(his) + -41.0 L-leucyl-tRNA(leu) + -22.0 L-lysyl-tRNA(lys) + -4.0 L-methionyl-tRNA(met) + -8.0 L-phenylalanyl-tRNA(phe) + 

In [13]:
#what does the result look like?
print('number of reactions immediately connected to target reaction:', len(immediately_connected))
print('number of reactions feeding into immediately connected reactions:', len(connected_by_one_reaction))

number of reactions immediately connected to target reaction: 4
number of reactions feeding into immediately connected reactions: 65
